In [ ]:
!python -m pip install --upgrade pip
%pip install pandas matplotlib seaborn scikit-learn openpyxl

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency, fisher_exact

## Intro

In [ ]:
PATH = 'C:/Users/aberti/Desktop/ProjectWork_AEQUITAS_AKKODIS/data/'
df = (
    pd.read_excel(PATH + 'Dataset_2.0_Akkodis.xlsx')
      .rename(columns=lambda c: c.lstrip().title())
)
df.head()

## Cleaning

In [ ]:
df = df.drop_duplicates(subset='Id', keep='last')
df.head()

In [ ]:
columns_to_drop = ['Id', 'Last Role', 'Year Of Insertion',
                   'Assumption Headquarters', 'Linked_Search__Key',
                   'Akkodis Headquarters']
THRESHOLD = 0.4

for col in df.columns:
  null_count = df[col].isna().sum() / df.shape[0]
  if null_count > THRESHOLD:
    columns_to_drop.append(col)
    print(f'DROPPED <Column: {col}> NULL count: {null_count*100:.2f}%')
  else:
    print(f'<Column: {col}> NULL count: {null_count*100:.2f}%')
  
df = df.drop(columns=columns_to_drop)
df.head()

In [ ]:
print(f'Columns that contain NaN values:\n {df.columns[df.isnull().any()].tolist()}')

for col in df.columns[df.isnull().any()].tolist():
  print(f'{col} values: {df[col].unique()} \n') # Analyze each NaN containing feature first to determine the default fill value

In [ ]:
fill_default = {
    'Residence': 'Not Specified',
    'Study Area': 'Not Specified',
    'Event_Type__Val': 'Not Specified',
}
df = df.fillna(fill_default)

In [ ]:
print(f'The remaining columns are:\n {df.columns}')
df.head()

## Feature Mapping 

###  **Candidate State**

In [ ]:
lookup = 'Candidate State'
distrib = Counter(df[lookup])
distrib_df = pd.DataFrame(distrib.items(), columns=[lookup, 'Count'])
distrib_df = distrib_df.sort_values(by='Count', ascending=False)
distrib_df.head(20).plot(x=lookup, y='Count', kind='bar', legend=False)
plt.title(lookup)
plt.xticks(rotation=45)

### **Age Range**

In [ ]:
lookup = 'Age Range'
custom_order = ['< 20 years', '20 - 25 years', '26 - 30 years', '31 - 35 years', '36 - 40 years', '40 - 45 years', '> 45 years']
df[lookup] = pd.Categorical(df[lookup], categories=custom_order, ordered=True).sort_values()

distrib = Counter(df[lookup])
distrib_df = pd.DataFrame(distrib.items(), columns=[lookup, 'Count'])
distrib_df.head(20).plot(x=lookup, y='Count', kind='bar', legend=False)
plt.title(lookup)
plt.xticks(rotation=45)

### **Residence**

In [ ]:
print(df['Residence'].unique())

In [ ]:
drop_keywords = [
    'ETHIOPIA',
    'SOUTH AFRICAN REPUBLIC',
    'USSR',
    'YUGOSLAVIA'
]

for kw in drop_keywords:
  mask = df['Residence'].str.contains(kw, na=False)
  df.loc[mask, 'Residence'] = f"{kw} » (STATE) ~ (FOREIGN)"
  print(f"Formatted '{kw}' as '{kw} » (STATE) ~ (FOREIGN)'")

In [ ]:
residence_list = df['Residence'].unique()
state_list = [s for s in residence_list 
              if ('(STATE)' in s) 
              or ('(COUNTRY)' in s)
              and not ('Not Specified' in s)
              ]
italy_list = [s for s in residence_list 
              if not ('(STATE)' in s) 
              and not ('(COUNTRY)' in s)
              and not ('Not Specified' in s)
              ]
state_list = sorted({s.split(' » ')[0] for s in state_list if ' » ' in s})
city_list = sorted({s.split(' » ')[0] for s in italy_list if ' » ' in s})
province_list = sorted({s.split(' » ')[1].split(' ~ ')[0] for s in italy_list if ' » ' in s and ' ~ ' in s})
region_list = sorted({s.split(' ~ ')[-1] for s in italy_list if ' ~ ' in s})

print(f"List of residence italian citys of the candidates in the dataset:\n {city_list}")
print(f"List of residence italian provinces of the candidates in the dataset:\n {province_list}")
print(f"List of residence italian regions of the candidates in the dataset:\n {region_list}")
print(f"List of residence states of the candidates in the dataset:\n {state_list}")

def map_residence_city(value):
    for city in city_list:
        if city in value:
            return city
    return 'Not Specified'

def map_residence_province(value):
    for prov in province_list:
        if prov in value:
            return prov
    return 'Not Specified'

def map_residence_region(value):
    for region in region_list:
        if region in value:
            return region
    return 'Not Specified'

def map_residence_state(value):
    for state in state_list:
        if state in value:
            return state
    return 'ITALY'

# Applicazione sulle colonne
df['Residence City']     = df['Residence'].apply(map_residence_city)
df['Residence Province'] = df['Residence'].apply(map_residence_province)
df['Residence Region']   = df['Residence'].apply(map_residence_region)
df['Residence State']    = df['Residence'].apply(map_residence_state)
df.head()


In [ ]:
european_countries = [
    'ALBANIA', 'AUSTRIA', 'BELARUS', 'BELGIUM', 'BULGARIA', 'CROATIA', 'CZECH REPUBLIC',
    'FRANCE', 'GERMANY', 'GREAT BRITAIN-NORTHERN IRELAND', 'GREECE', 'ITALY', 'LATVIA',
    'LITHUANIA', 'LUXEMBOURG', 'MALTA', 'MOLDOVA', 'MONACO', 'MONTENEGRO', 'NETHERLANDS',
    'NORWAY', 'POLAND', 'PORTUGAL', 'ROMANIA', 'RUSSIA', 'SAN MARINO', 'SERBIA', 'SLOVAKIA',
    'SLOVENIA', 'SPAIN', 'SWEDEN', 'SWITZERLAND', 'UKRAINE'
]
df['European Residence'] = df['Residence State'].apply(lambda x: 'Yes' if x in european_countries else 'No')
df['Italian Residence'] = df['Residence State'].apply(lambda x: 'Yes' if 'ITALY' in x else 'No')
df.head()

In [ ]:
lookup = 'Residence State'
distrib = Counter(df[df[lookup] != 'ITALY'][lookup])
distrib_df = pd.DataFrame(distrib.items(), columns=[lookup, 'Count'])
distrib_df = distrib_df.sort_values(by='Count', ascending=False)
distrib_df.head(20).plot(x=lookup, y='Count', kind='bar', legend=False)
plt.title(lookup + ' (excluding ITALY)')

In [ ]:
lookup = 'Residence Region'
distrib = Counter(df[lookup])
distrib_df = pd.DataFrame(distrib.items(), columns=[lookup, 'Count'])
distrib_df = distrib_df.sort_values(by='Count', ascending=False)
distrib_df.head(20).plot(x=lookup, y='Count', kind='bar', legend=False)
plt.title(lookup)

In [ ]:
lookup = 'European Residence'
distrib = Counter(df[lookup])
distrib_df = pd.DataFrame(distrib.items(), columns=[lookup, 'Count'])
distrib_df = distrib_df.sort_values(by='Count', ascending=False)
distrib_df.head(20).plot(x=lookup, y='Count', kind='bar', legend=False)
plt.title(lookup)
plt.xticks(rotation=45)

lookup = 'Italian Residence'
distrib = Counter(df[lookup])
distrib_df = pd.DataFrame(distrib.items(), columns=[lookup, 'Count'])
distrib_df = distrib_df.sort_values(by='Count', ascending=False)
distrib_df.head(20).plot(x=lookup, y='Count', kind='bar', legend=False)
plt.title(lookup)
plt.xticks(rotation=45)

In [ ]:
df = df.drop(columns=['Residence'])
df.head()

### **Sex**

In [ ]:
lookup = 'Sex'
distrib = Counter(df[lookup])
distrib_df = pd.DataFrame(distrib.items(), columns=[lookup, 'Count'])
distrib_df = distrib_df.sort_values(by='Count', ascending=False)
distrib_df.head(20).plot(x=lookup, y='Count', kind='bar', legend=False)
plt.title(lookup)
plt.xticks(rotation=45)

### **Study Area**

In [ ]:
lookup = 'Study Area'
distrib = Counter(df[lookup])
distrib_df = pd.DataFrame(distrib.items(), columns=[lookup, 'Count'])
distrib_df = distrib_df.sort_values(by='Count', ascending=False)
distrib_df.head(20).plot(x=lookup, y='Count', kind='bar', legend=False)
plt.title(lookup)

### **Study Title**

In [ ]:
lookup = 'Study Title'
distrib = Counter(df[lookup])
distrib_df = pd.DataFrame(distrib.items(), columns=[lookup, 'Count'])
distrib_df = distrib_df.sort_values(by='Count', ascending=False)
distrib_df.head(20).plot(x=lookup, y='Count', kind='bar', legend=False)
plt.title(lookup)
plt.xticks(rotation=45)

### **Years Experience**

In [ ]:
lookup = 'Years Experience'
custom_order = ['[0]', '[0-1]', '[1-3]', '[3-5]', '[5-7]', '[7-10]', '[+10]']
df[lookup] = pd.Categorical(df[lookup], categories=custom_order, ordered=True).sort_values()

distrib = Counter(df[lookup])
distrib_df = pd.DataFrame(distrib.items(), columns=[lookup, 'Count'])
distrib_df.head(20).plot(x=lookup, y='Count', kind='bar', legend=False)
plt.title(lookup)
plt.xticks(rotation=45)

### **Event_type__val**

In [ ]:
lookup = 'Event_Type__Val'
distrib = Counter(df[lookup])
distrib_df = pd.DataFrame(distrib.items(), columns=[lookup, 'Count'])
distrib_df = distrib_df.sort_values(by='Count', ascending=False)
distrib_df.head(20).plot(x=lookup, y='Count', kind='bar', legend=False)
plt.title(lookup)

## Data Visualization

### **Sex and Candidate State**

In [ ]:
pivot = df.pivot_table(index='Sex', columns='Candidate State', aggfunc='size', fill_value=0)
pivot.plot(kind='bar', figsize=(10, 6))
plt.title('Candidate State by Sex')
plt.xticks(rotation=45)
plt.legend(title='Candidate State', loc='upper left')

pivot = df.pivot_table(index='Sex', columns='Candidate State', aggfunc='size', fill_value=0)
pivot_percentage = pivot.div(pivot.sum(axis=1), axis=0)
pivot_percentage.plot(kind='bar', stacked=True, figsize=(10, 6))
plt.title('Candidate State by Sex (Normalized)')
plt.xticks(rotation=45)
plt.legend(title='Candidate State', loc='upper left')

### **Age Range and Candidate State**

In [ ]:
sns.histplot(
    data= df,
    x='Age Range',
    hue='Candidate State',
    multiple='stack',
    palette='Set3',
    legend=True
)
plt.title("Distribution of Age Ranges by Candidate State")
plt.xticks(rotation=45)

In [ ]:
sns.boxplot(
    data=df,
    x='Candidate State',
    y=df['Age Range'].map(lambda x: int(x.split('-')[0]) if '-' in x else (19 if '<' in x else 46)),
    palette='Set3',
    hue='Candidate State',
    legend=True
)
plt.title("Candidate State by Age Range (Numerical Approximation)", fontsize=14)
plt.xticks(rotation=45)


### **Age Range Correlation**

In [ ]:
df_encoded = df.copy()

custom_order = ['< 20 years', '20 - 25 years', '26 - 30 years', '31 - 35 years', '36 - 40 years', '40 - 45 years', '> 45 years']

df_encoded['Age Range'] = pd.Categorical(
    df_encoded['Age Range'],
    categories=custom_order,
    ordered=True
).codes + 1

cat_cols = df_encoded.columns.drop('Age Range')

enc = LabelEncoder()
for col in df_encoded.columns.drop('Age Range'):
    if col != 'Age Range':
        df_encoded[col] = enc.fit_transform(df_encoded[col])

correlation_matrix = df_encoded.corr()

sns.heatmap(correlation_matrix, annot=True, fmt=".1f")
plt.title('Correlation Matrix')

## Chi-squared Tests

### Candidate State

In [ ]:
df['Hired_Flag'] = df['Candidate State'] == 'Hired'

# Tabelle di contingenza
contingency_sex    = pd.crosstab(df['Sex'], df['Hired_Flag'])
contingency_age    = pd.crosstab(df['Age Range'], df['Hired_Flag'])
contingency_region = pd.crosstab(df['Residence Region'], df['Hired_Flag'])

# Chi-squared tests
tables = {
    'Sex': contingency_sex,
    'Age Range': contingency_age,
    'Residence Region': contingency_region
}
for var, table in tables.items():
    chi2, p, dof, expected = chi2_contingency(table, correction=False)
    test_name = 'Chi-squared'
    
    # se 2×2 e attese <5 → Fisher’s exact
    if table.shape == (2,2) and (expected < 5).any():
        _, p = fisher_exact(table)
        test_name = "Fisher's exact"
    
    n = table.values.sum()
    k = min(table.shape)
    cramer_v = np.sqrt(chi2 / (n * (k-1)))
    
    # Stampa a video
    print(f"--- {var} ---")
    print("Expected frequencies:")
    print(pd.DataFrame(expected, index=table.index, columns=table.columns))
    print()
    print(f"{test_name}: χ² = {chi2:.2f}, p = {p:.3f}, dof = {dof}, Cramér’s V = {cramer_v:.3f}")
    print("Conclusion: Significant association between two variables (Dependent)" if p < 0.05 else "Conclusion: No significant association between two variables (Independent)")
    print()
    

### Candidate State vs Sex

In [ ]:
selection_order = [
    'Imported', 'In selection', 'First contact',
    'QM', 'Vivier', 'Economic proposal', 'Hired'
]
lookout = 'Sex'
contingency_tables = {}

for i, state in enumerate(selection_order):
    post_states = selection_order[i+1:]
    if post_states:
        df_state      = df[df['Candidate State'] == state]
        df_post_state = df[df['Candidate State'].isin(post_states)]
        contingency_tables[state] = pd.DataFrame({
            f'Post {state}': df_post_state.groupby(lookout, observed=True).size(), state: df_state.groupby(lookout, observed=True).size()
        }).fillna(0).astype(int)

for var, table in contingency_tables.items():
    chi2, p, dof, expected = chi2_contingency(table, correction=False)
    test_name = 'Chi-squared'
    
    # se 2×2 e attese <5 → Fisher’s exact
    if table.shape == (2,2) and (expected < 5).any():
        _, p = fisher_exact(table)
        test_name = "Fisher's exact"
    
    n = table.values.sum()
    k = min(table.shape)
    cramer_v = np.sqrt(chi2 / (n * (k-1)))
    
    # Stampa a video
    print(f"--- {var} ---")
    print(f"Contingency table (by {lookout}):")
    print(table)
    print()
    print("Expected frequencies:")
    print(pd.DataFrame(expected, index=table.index, columns=table.columns))
    print()
    print(f"{test_name}: χ² = {chi2:.2f}, p = {p:.3f}, dof = {dof}, Cramér’s V = {cramer_v:.3f}")
    print("Conclusion: Significant association between two variables (Dependent)" if p < 0.05 else "Conclusion: No significant association between two variables (Independent)")
    print()

### Candidate State vs Age Range

In [ ]:
selection_order = [
    'Imported', 'In selection', 'First contact',
    'QM', 'Vivier', 'Economic proposal', 'Hired'
]
lookout = 'Age Range'
contingency_tables = {}

for i, state in enumerate(selection_order):
    post_states = selection_order[i+1:]
    if post_states:
        df_state      = df[df['Candidate State'] == state]
        df_post_state = df[df['Candidate State'].isin(post_states)]
        contingency_tables[state] = pd.DataFrame({
            f'Post {state}': df_post_state.groupby(lookout, observed=True).size(), state: df_state.groupby(lookout, observed=True).size()
        }).fillna(0).astype(int)

for var, table in contingency_tables.items():
    chi2, p, dof, expected = chi2_contingency(table, correction=False)
    test_name = 'Chi-squared'
    
    # se 2×2 e attese <5 → Fisher’s exact
    if table.shape == (2,2) and (expected < 5).any():
        _, p = fisher_exact(table)
        test_name = "Fisher's exact"
    
    n = table.values.sum()
    k = min(table.shape)
    cramer_v = np.sqrt(chi2 / (n * (k-1)))
    
    # Stampa a video
    print(f"--- {var} ---")
    print(f"Contingency table (by {lookout}):")
    print(table)
    print()
    print("Expected frequencies:")
    print(pd.DataFrame(expected, index=table.index, columns=table.columns))
    print()
    print(f"{test_name}: χ² = {chi2:.2f}, p = {p:.3f}, dof = {dof}, Cramér’s V = {cramer_v:.3f}")
    print("Conclusion: Significant association between two variables (Dependent)" if p < 0.05 else "Conclusion: No significant association between two variables (Independent)")
    print()

### Candidate State vs Residence Region

In [ ]:
selection_order = [
    'Imported', 'In selection', 'First contact',
    'QM', 'Vivier', 'Economic proposal', 'Hired'
]
lookout = 'Residence Region'
contingency_tables = {}

for i, state in enumerate(selection_order):
    post_states = selection_order[i+1:]
    if post_states:
        df_state      = df[(df['Candidate State'] == state) & (df[lookout] != 'Not Specified')] # Excluding Not Specified
        df_post_state = df[(df['Candidate State'].isin(post_states)) & (df[lookout] != 'Not Specified')] # Excluding Not Specified
        contingency_tables[state] = pd.DataFrame({
            f'Post {state}': df_post_state.groupby(lookout, observed=True).size(), state: df_state.groupby(lookout, observed=True).size()
        }).fillna(0).astype(int)

for var, table in contingency_tables.items():
    chi2, p, dof, expected = chi2_contingency(table, correction=False)
    test_name = 'Chi-squared'
    
    # se 2×2 e attese <5 → Fisher’s exact
    if table.shape == (2,2) and (expected < 5).any():
        _, p = fisher_exact(table)
        test_name = "Fisher's exact"
    
    n = table.values.sum()
    k = min(table.shape)
    cramer_v = np.sqrt(chi2 / (n * (k-1)))
    
    # Stampa a video
    print(f"--- {var} ---")
    print(f"Contingency table (by {lookout}):")
    print(table)
    print()
    print("Expected frequencies:")
    print(pd.DataFrame(expected, index=table.index, columns=table.columns))
    print()
    print(f"{test_name}: χ² = {chi2:.2f}, p = {p:.3f}, dof = {dof}, Cramér’s V = {cramer_v:.3f}")
    print("Conclusion: Significant association between two variables (Dependent)" if p < 0.05 else "Conclusion: No significant association between two variables (Independent)")
    print()